# Example of working with GHCN data from Romania

[Global historcal climate network](http://www.ncdc.noaa.gov/ghcnm/) weather station data from Romania.

Requires Pandas version > 0.19.0

In [ ]:
# Load some libraries
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import *
from colorama import Fore
matplotlib.style.use('ggplot')
%matplotlib inline

## Step 1: data preperation

From our Station Data files we need to create:
* one single data structure
* Date indexed data (with only one index for all the datasets)
* Station names as column identifiers

We will use the [Pandas library](http://pandas.pydata.org/pandas-docs/stable/index.html), as it is perfectly suited to our task. We have read it in above with the alias **pd**.

Before we try and read all the data, let's test a procedure with one single station file.

In [ ]:
# Read a station data file with Pandas
test_data = pd.read_csv("Data/station_data/BUM00015502_VIDIN_BU_.csv")
test_data.head()

Looks good, but the Dates should be an index, not a column, and they should also be a date object, not a simple integer (we get much more functionality that way).

In [ ]:
# Make a list of datetime values out of the integer dates using a list comprehension technique
dates = []
for date in test_data['DATE']:
    dates.append(pd.datetime.strptime(str(date),"%Y%m%d"))
# The above could have been done more effectivley using list comprehension

# Next set the new list as an index, and remove the old column from the dataset
test_data.index = dates
test_data = test_data.drop(['DATE','PRCP'], axis=1)

test_data.head()

Great! We can plot a simple preview of the data to make sure it looks good.

In [ ]:
test_data.plot()

The preview plot is messy as our data are not contiguous, but as a quick-check, it seems like everything is more-or-less fine.

So, reading a single file is easy, and straightforward. But we want to do some exploratory analysis on multiple station measurements. For this we will need to read all the station data together into a consistent data object. 

In [ ]:
# Make a small tools (functions) to help with the work

def station_name(fname):
    """Return the station ID from a path/filename.csv string"""
    tmp = fname.split('/')[-1]
    return tmp.split('_')[0]

In [ ]:
# If we were on a Mac or Linux system, we could get the file list via a bash command
flist = !ls Data/station_data/*.csv

In [ ]:
# But this will break on windows. To make our code cross-platform we use a python
# library to find all the files instead. This is much better than hard-coding the files!

frames = [] # an empty list to hold each data object as it is loaded

mypath = 'Data/station_data/'          # Set path to data
for item in tqdm(os.listdir(mypath)):        # Find all files in that path and loop over them
    if '.csv' in item:                 # If the file is a csv type do something...
        fname = ''.join([mypath,item])
        station = station_name(fname)
        #print('Reading data from station', station)
        tmp = pd.read_csv(fname)
        dates = [pd.datetime.strptime(str(date),"%Y%m%d") for date in tmp['DATE']]
        tmp.index = dates
        tmp = tmp.drop(['DATE','PRCP'], axis=1) # get rid of date and precipitation columns
        tmp.columns = [station]     # Re-name TAVG to be the station name
        frames.append(tmp)
print("{0} GHCN files read".format(len(frames)))

In [ ]:
df = pd.concat(frames, axis=1)  # Join all the seperate data together into one object

## Step 2: Cleaning the dataset for analysis

Now we have created a dataframe **df** holding all the station data with one coherant time index.

This abstraction will do much of the work for us...

In [ ]:
# First lets see how long these data run for in time
print("minimum date:", min(df.index).date())
print("maximum date:", max(df.index).date())

In [ ]:
# Now let's look at a statistical description of these data
df.describe()

There is a clear problem with these stats. Most of these data seem to have a missing value of `-9999.0` included.
To proceede we should replace with with a missing data type that we can operate with `np.nan`

In [ ]:
# we can replace all -999.0 values with np.nan like this
df[df == -9999.0] = np.nan

In [ ]:
# Now, the dataframe values seems reasonable, except we can see there are many series which are empty.
# They were just full of missing values for whatever reason.

df.describe()

In [ ]:
# It looks like we can simply filter out data that now has a low count (e.g. < 10,000).

limit = 0

for key in df:
    if df[key].count() <= limit:
        print('removing', key,'from df object.')
        df = df.drop([key], axis=1)

In [ ]:
# Much better! Finally a clean df object, that we can work from.

df.describe()

## Step 3. Creating a mean time series


In [ ]:
for key in df:
    plt.plot(df[key], lw=0.3, alpha=0.75)
plt.title('Individual stations')
plt.show()

Aggregated, these data gives a clearer picture

In [ ]:
df.mean(axis=1).plot(title='Romanian TAVG', lw=0.5, alpha=0.8)

Looking at the data it is easy to see the seasonal signal is dominant. One method of removing the seasonal signal and work with anomalies is to subtract a day-of-the-year (DOY) mean.

In [ ]:
# make the average data a new series, and strip out nan values for working ease
df_mean = pd.DataFrame(df.mean(axis=1), columns=['mean_temp'])  # make a new df object
#df_mean = df_mean[df_mean.notnull().values]                     # remove missing values

I can look at every day in the time series, find its DOY value (an integer from 1 - 366), make a corresponding list, and use it to subscript the mean dataframe, and find the mean on each day of the year. I place these values in a dictionary with integer doy as a key, for ease of use later when deseasonalising.  

In [ ]:
doy_index = np.array([ date.dayofyear for date in df_mean.index])  # make an array of DOY values to use as a lookup
d = {}
for doy in range(1,367):
    d[doy] = df_mean[doy_index == doy].mean().values[0]

In [ ]:
# Veryify it looks okay...
tmp = []
for key in d:
    tmp.append(d[key])
plt.plot(range(1,367), tmp)
plt.title("Seasonal Temperature signal in Romania")
plt.xlabel('DOY')
plt.xlim([1,367])
plt.ylabel("Temp. °C")
plt.show()

## Step 4: Remove the seasonal signal from these data

In [ ]:
dszn = []
for day in df_mean.index:
    dszn.append(df_mean['mean_temp'][day] - d[day.dayofyear])
dszn = np.array(dszn)

df_mean['anom'] = dszn

In [ ]:
# All of the above could have just been done in one line...
# df_mean['anom'] = np.array([df_mean['mean_temp'][day] - d[day.dayofyear] for day in df_mean.index])

In [ ]:
plt.plot(df_mean.index, df_mean.anom, lw=0.5, alpha = 0.75)
plt.title(r"Romanina $\delta$ Temp. °C")
plt.show()

## Step 5: Using the data for something useful!

Based on historical Romanian average temperature anomalies, how does a given value rank?

Requires an average temperature and a date as input.

In [ ]:
def anom_of_given_date(date, temperature):
    """Given a specific date and temperature, return the delta temp value"""
    qtmp =pd.DataFrame([temperature], index=[pd.datetime.strptime(str(date),"%Y%m%d")])
    qanom = temperature - d[qtmp.index[0].dayofyear]
    print("Day of year for {0} is {1}".format(date, qtmp.index[0].dayofyear))
    print("𝛿 temp. = {0:3.2f}°C".format(qanom))
    return qanom

In [ ]:
anom = anom_of_given_date(date=19830901, temperature=10)

In [ ]:
a, bins, c = plt.hist(df_mean.anom, bins=80, normed=True)
plt.title(r"Romanian $\delta$ temperature")
plt.xlabel(r"$\delta$ Temp. °C")
plt.ylim(0,0.13)
plt.vlines(anom, 0, 0.16, label="day to check")

y = matplotlib.mlab.normpdf(bins, df_mean.anom.mean(), df_mean.anom.std())
plt.plot(bins, y, label='Gaussian pdf')

plt.legend()
plt.show()

Let's see that in a cumulative density plot as well

In [ ]:
a,b,c = plt.hist(df_mean.anom, bins=100, cumulative=True, normed=True)
plt.vlines(anom, 0, 1.1)
plt.ylim(0,1)
plt.title(r"Cumulative $\delta$ distribution and value to test")
plt.show()

Rank the anomaly values, and count how many times our value to check is larger than the ranked values. Divide this by the total number of values to get the percentile position.

In [ ]:
ranked = np.array(sorted(df_mean.anom.values))
print("𝛿 of {0:3.2f}°C has a percentile rank of {1:3.2f}".format(anom, len(ranked[anom > ranked]) / len(ranked)))

### Finally, combine the above into a function that can be re-run easily

In [ ]:
def check_anomaly(date, temperature):
    anom = anom_of_given_date(date=date, temperature=temperature)
    a,b,c = plt.hist(df_mean.anom, bins=100, cumulative=True, normed=True)
    plt.vlines(anom, 0, 1.1)
    plt.ylim(0,1)
    plt.title(r"Cumulative $\delta$ distribution")
    plt.ylabel('Density')
    plt.xlabel(r"$\delta$ T. [°C]")
    plt.show()
    ranked = np.array(sorted(df_mean.anom.values))
    print("𝛿 of {0:3.2f}°C has a ".format(anom),end="")
    print(Fore.RED + "percentile rank of {0:3.2f}".format(len(ranked[anom > ranked]) / len(ranked)))

In [ ]:
check_anomaly(date=20161020, temperature=15)